# Scan

In [ ]:
import sys
import pandas as pd
import requests, urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from insightvm_authen import InsightvmAuthentication

In [ ]:
HOSTNAME = InsightvmAuthentication().getHostname()
HEADER = InsightvmAuthentication().getRequestHeader()
# HOSTNAME, HEADER

# Get All Available Scan Engine Pool

In [ ]:
def getAllAvailableScanEnginePool():
    getAllAvailableScanEngineURL = HOSTNAME + "api/3/scan_engine_pools"

    print("Hostname : " + getAllAvailableScanEngineURL)
    r = requests.get(getAllAvailableScanEngineURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r["resources"]
    else:
        print(r.status_code)

In [ ]:
def extractScanEnginePool(data={}):
    return {
        "poolId": data["id"],
        "poolName": data["name"],
        "engines": data["engines"] if "engines" in data else [],
        "sites": data["sites"] if "sites" in data else [],
    }

In [ ]:
availableScanEnginePool = getAllAvailableScanEnginePool()
availableScanEnginePool = [ extractScanEnginePool(data=pool) for pool in availableScanEnginePool ]

In [ ]:
pd.DataFrame(availableScanEnginePool)

# Get All Available Scan Engine

In [ ]:
def getAllAvailableScanEngine():
    getAllAvailableScanEngineURL = HOSTNAME + "api/3/scan_engines"

    print("Hostname : " + getAllAvailableScanEngineURL)
    r = requests.get(getAllAvailableScanEngineURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r["resources"]
    else:
        print(r.status_code)

In [ ]:
def extractScanEngine(data={}):
    return {
        "engineId": data["id"],
        "engineName": data["name"],
        "engineStatus": data["status"],
        "engineIP": data["address"],
        "enginePort": data["port"],
    }

In [ ]:
availableScanEngine = getAllAvailableScanEngine()
availableScanEngine = [ extractScanEngine(data=engine) for engine in availableScanEngine ]

In [ ]:
pd.DataFrame(availableScanEngine)

# Get Scan Engine Name

In [ ]:
def getScanEngineName(engineId=""):
    getScanEngineNameURL = HOSTNAME + "api/3/scan_engines/" + str(engineId)

    print("\nGetting Scan Engine Name")
    print("Hostname : " + getScanEngineNameURL)
    r = requests.get(getScanEngineNameURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        if "name" in r:
            return r["name"]
        else:
            print("[Error] Not Found Scan Engine")
            sys.exit()
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
engineId = "1"
getScanEngineName(engineId=engineId)

## Get SiteID by Scan Engine ID

In [ ]:
def getSiteIDByScanEngineID(engineId="", page=None):
    getSiteIDByScanEngineIDURL = HOSTNAME + "api/3/scan_engines/" + str(engineId) + "/sites?page=" + str(page)
    
    params = {
        "size" : "500"
    }
    
    print("\nGetting SiteID by Scan Engine ID")
    print("Hostname : " + getSiteIDByScanEngineIDURL)
    r = requests.get(getSiteIDByScanEngineIDURL, params=params, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
def extractSiteInfo(data={}):
    return {
        "scanEngine" : str(data["scanEngine"]),
        "scanTemplateID" : data["scanTemplate"],
        "importance" : data["importance"],
        "siteID" : data["id"],
        "siteName" : data["name"],
        "assets" : data["assets"]
    }

In [ ]:
engineId = 13
siteInfos = getSiteIDByScanEngineID(engineId=engineId, page=0)
siteInfos["page"], len(siteInfos["resources"])

In [ ]:
pd.DataFrame([extractSiteInfo(data=site) for site in siteInfos["resources"]])

### Move All Site in Scan Engine to Scan Engine Pools Lab

In [ ]:
def getAllSiteInfoByScanEngineID(engineId=None):
    siteInfos = []
    
    page = 0
    totalPages = 0
    totalResources = 0
    
    if engineId:
        while True:
            result = getSiteIDByScanEngineID(engineId=str(engineId), page=page)
            
            siteInfos += [extractSiteInfo(data=site) for site in result["resources"]]
            
            if page == 0:
                totalPages = result["page"]["totalPages"]
                totalResources = result["page"]["totalResources"]
                print("Engine Id : " + str(engineId) + ", Total Pages : " + str(totalPages) + ", Total Resources : " + str(totalResources))
            
                if len(siteInfos) < 500: 
                    print("[Error] Query Size(500) != Retrieve Size[SiteInfos] from Page 0(" + str(len(siteInfos)) + ")")
                    return siteInfos
                    
            if totalPages == page and totalResources == len(siteInfos):
                print("[Getting Complete] SiteInfos Length : " + str(len(siteInfos)))
                return siteInfos
            else:
                print("")
                page += 1
    


In [ ]:
engineIds = [1,2,3,4]

In [ ]:
allSiteInfoByEngineIds = []
for engineId in engineIds:
    allSiteInfoByEngineIds += getAllSiteInfoByScanEngineID(engineId=str(engineId))

In [ ]:
allSiteInfoByEngineIds_df = pd.DataFrame(allSiteInfoByEngineIds)
allSiteInfoByEngineIds_df

In [ ]:
filename = "outputs/" + "siteInfo_Engine.xlsx"
allSiteInfoByEngineIds_df.to_excel(filename, index=False, header=True)

In [ ]:
def getSiteScanConfig(siteId=""):
    getSiteScanConfigURL = HOSTNAME + "api/3/sites/" + str(siteId)
    
    print("\nGet Site Scan Configuration by Site ID")
    print("Hostname : " + getSiteScanConfigURL)
    r = requests.get(getSiteScanConfigURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
def updateSiteScanConfig(siteId="", payload={}):
    updateSiteScanConfigURL = HOSTNAME + "api/3/sites/" + str(siteId)
    
    print("\nUpdate Site Scan Configuration by Site ID")
    print("Hostname : " + updateSiteScanConfigURL)
    r = requests.put(updateSiteScanConfigURL, headers=HEADER, json=payload, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
def updateAllSiteScanConfig(siteinfo={}):
#     getSiteScanConfig(siteId=siteinfos["siteID"])
    poolID = 1
    
    payload = {
        "engineId": str(poolID),
        'importance': siteinfo["importance"],
        'name': siteinfo["siteName"],
        'scanTemplateId': siteinfo["scanTemplateID"],
    }
    updateSiteScanConfig(siteId=siteinfo["siteID"], payload=payload)

#     getSiteScanConfig(siteId=siteId)
    

In [ ]:
for siteinfo in allSiteInfoByEngineIds:
    print("\n+
          ")
    print(siteinfo)
    updateAllSiteScanConfig(siteinfo=siteinfo)
    print(extractSiteInfo(data=getSiteScanConfig(siteId=siteinfo["siteID"])))

In [ ]:
getSiteScanConfig(siteId="17")

# Get All Scan Templates

In [ ]:
def getAllScanTemplate():
    getAllScanTemplateURL = HOSTNAME + "api/3/scan_templates"

    print("Hostname : " + getAllScanTemplateURL)
    r = requests.get(getAllScanTemplateURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r["resources"]
    else:
        print(r.status_code)

In [ ]:
def extractScanTemplate(data={}):
    return {
        "scanTemplateId": data["id"],
        "scanTemplateName": data["name"],
    }

In [ ]:
getAllScanTemplate = getAllScanTemplate()
getAllScanTemplate = [extractScanTemplate(data=template) for template in getAllScanTemplate]

In [ ]:
getAllScanTemplate

In [ ]:
getAllScanTemplate_df = pd.DataFrame(getAllScanTemplate)
getAllScanTemplate_df

In [ ]:
getAllScanTemplate_df[getAllScanTemplate_df["scanTemplateName"].str.contains("Test")]

In [ ]:
getAllScanTemplate_df[getAllScanTemplate_df["scanTemplateName"].str.contains("(Full)|Server")]

In [ ]:
getAllScanTemplate_df[getAllScanTemplate_df["scanTemplateId"].str.contains('pci-internal-audit-_-cis-policy-_enhanced-log_')]

In [ ]:
def getScanTemplateName(templateId=""):
    getScanTemplateNameURL = HOSTNAME + "api/3/scan_templates/" + str(templateId)

    print("\nGetting Scan Template Name")
    print("Hostname : " + getScanTemplateNameURL)
    r = requests.get(getScanTemplateNameURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        if "name" in r:
            return r["name"]
        else:
            print("[Error] Not Found Scan Template")
            sys.exit()
    else:
        print("[Error] Request Connection Error.")
        print(r.status_code)

In [ ]:
templateId = "pci-internal-audit-_-cis-policy-_enhanced-log_"
getScanTemplateName(templateId=templateId)

# Manage Scan Status

status : "pause", "stop", "resume"

In [ ]:
def manageScanStatus(scanId="", status=""):
    manageScanStatusURL = HOSTNAME + "api/3/scans/" + str(scanId) + "/" + str(status)

    print("Hostname : " + manageScanStatusURL)
    r = requests.post(manageScanStatusURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        print("[Successful] Update Scan Status")
    else:
        print(r.status_code)

In [ ]:
scanId = 12345
status = "pause"

In [ ]:
manageScanStatus(scanId=scanId, status=status)

# Get Scan Information By ID

In [ ]:
def getScanInfoById(scanId=""):
    getScanByIDURL = HOSTNAME + "api/3/scans/" + str(scanId)

    print("Hostname : " + getScanByIDURL)
    r = requests.get(getScanByIDURL, headers=HEADER, verify=False)
    if r.status_code == 200:
        r = r.json()
        return r
    else:
        print(r.status_code)

In [ ]:
scanId = 12345

In [ ]:
scanInfo = getScanInfoById(scanId=scanId)
scanInfo